In [60]:
import numpy as np
import pandas as pd
import requests as rq
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [61]:
from sagemaker.session import Session
from sagemaker.experiments.run import Run, load_run 
from sagemaker.utils import unique_name_from_base
import sagemaker
import logging
import sys
import os
import boto3
import datetime as dt
import re

In [62]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [63]:
experiment_name = unique_name_from_base("weatherApp-training-")
run_name = "experiment-train-run"


In [64]:
print(experiment_name)

weatherApp-training--1692051401-5cd4


In [65]:
weather_types = { 'NA': 'Not available', '-1': 'Trace rain', '0': 'Clear night', '1': 'Sunny day', '2': 'Partly cloudy (night)', '3': 'Partly cloudy (day)', '4': 'Not used', '5': 'Mist', '6': 'Fog', '7': 'Cloudy', '8': 'Overcast', '9': 'Light rain shower (night)', '10': 'Light rain shower (day)', '11': 'Drizzle', '12': 'Light rain', '13': 'Heavy rain shower (night)', '14': 'Heavy rain shower (day)', '15': 'Heavy rain', '16': 'Sleet shower (night)', '17': 'Sleet shower (day)', '18': 'Sleet', '19': 'Hail shower (night)', '20': 'Hail shower (day)', '21': 'Hail', '22': 'Light snow shower (night)', '23': 'Light snow shower (day)', '24': 'Light snow', '25': 'Heavy snow shower (night)', '26': 'Heavy snow shower (day)', '27': 'Heavy snow', '28': 'Thunder shower (night)', '29': 'Thunder shower (day)', '30': 'Thunder'}

visibility_dict = { 'UN': 'Unknown', 'VP': 'Very poor - Less than 1 km', 'PO': 'Poor - Between 1-4 km', 'MO': 'Moderate - Between 4-10 km', 'GO': 'Good - Between 10-20 km', 'VG': 'Very good - Between 20-40 km', 'EX': 'Excellent - More than 40 km'}
 

In [66]:
def fetch_api_data():
    """
    Fetching the data from the MetOffice API.
    
    Args:
        None
    Returns: 
        JSON object
    Raises:
        None
    """
    data_url = "http://datapoint.metoffice.gov.uk/public/data/val/wxobs/all/json/3772?res=hourly&key=12a3f254-6c4a-4f6d-a464-a7a2290a7226"
    response = rq.get(data_url)
    return response.json()


In [67]:
def get_location(data):
    """
    Returns the location of the weather data.
    
    Args:
        data (JSON): The weather data.
    Returns:
        str: The location of the weather data.
    Raises:
        KeyError: If the location is not found in the data.
    """
    return f"{data['SiteRep']['DV']['Location']['name']}, {data['SiteRep']['DV']['Location']['country']}"

In [68]:
def make_dataframe(data):
    """
    Converts the JSON data into a pandas dataframe.
    
    Args:
        data (JSON): JSON data from the Met Office DataPoint API
    Returns:
        weather_data (pandas dataframe): dataframe containing the weather data
    Raises:
        None
    """

    weather_data = None
    for day in data['SiteRep']['DV']['Location']['Period']:
        df = pd.DataFrame(day['Rep'])
        df['date'] = day['value']
        weather_data = pd.concat([weather_data, df], ignore_index=True)
    return weather_data

In [69]:
def clean_dataframe(weather_data):
    """
    Applies data manipulations to the weather data and prepare dataframe.
    
    Args:
        weather_data (DataFrame): weather data
    Returns:
        weather_data (DataFrame): weather data after manipulations
    Raises:
        None
    """
    headers = {'D':'Wind Direction(compass)', 'G':'Wind Gust(mph)', 'H':'Screen Relative Humidity(%)', 'P':'Pressure(hpa)', 'S':'Wind Speed(mph)', 'T':'Temperature(C)', 'V':'Visibility(m)', 'W':'Weather Type', 'Pt':'Pressure Tendency', 'Dp':'Dew Point(C)', '$':'Minutes Since 12o Clock'}
    weather_data.rename(columns=headers, inplace=True)
    weather_data.dropna(how='any', inplace=True)
    compass_directions_map = { 'N': 1, 'NNE':2, 'NE': 3, 'ENE':4, 'E': 5, 'ESE':6, 'SE': 7, 'SSE':8, 'S': 9, 'SSW':10, 'SW': 11, 'WSW':12, 'W': 13, 'WNW':14, 'NW': 15, 'NNW':16}
    weather_data['Wind Direction(compass)'] = weather_data['Wind Direction(compass)'].map(compass_directions_map)
    Pressure_tendency_map = { 'F': 0, 'R': 1, 'S': 2}   
    weather_data['Pressure Tendency'] = weather_data['Pressure Tendency'].map(Pressure_tendency_map)
    weather_data['date'] = pd.to_datetime(weather_data['date'], format='%Y-%m-%dZ')
    column_datatypes = {'Wind Direction(compass)':int, 'Wind Gust(mph)':float, 'Screen Relative Humidity(%)':float, 'Pressure(hpa)':float, 'Wind Speed(mph)':float, 'Temperature(C)':float, 'Visibility(m)':float, 'Weather Type':int, 'Pressure Tendency':int, 'Dew Point(C)':float, 'Minutes Since 12o Clock':int}
    weather_data = weather_data.astype(column_datatypes)
    weather_data['day'] = weather_data['date'].dt.day
    weather_data['month'] = weather_data['date'].dt.month
    weather_data['year'] = weather_data['date'].dt.year
    weather_data.drop('date', axis=1, inplace=True)
    return weather_data

In [70]:
def prepare_model_data(weather_data):
    """This function prepares the data for the model by sorting the data by date and time and then shifting the output columns by 1 row.
    
    args:
        weather_data: The dataframe containing the weather data.
    returns:
        A dataframe containing the weather data with the output columns shifted by 1 row.
    raises:
        None
    """
    
    weather_data.sort_values(by=['year','month','day', 'Minutes Since 12o Clock'], kind='mergesort', inplace=True)
    output = weather_data[['Wind Direction(compass)', 'Pressure(hpa)', 'Wind Speed(mph)', 'Temperature(C)', 'Visibility(m)', 'Weather Type']].shift(-1)
    output.columns = ['Wind Direction(compass) (t+1)', 'Pressure(hpa) (t+1)', 'Wind Speed(mph) (t+1)', 'Temperature(C) (t+1)', 'Visibility(m) (t+1)', 'Weather Type (t+1)']
    data_for_model = pd.concat([weather_data, output], axis=1)
    data_for_model.dropna(inplace=True)
    return data_for_model
    

In [71]:
data = fetch_api_data()
weather_data = make_dataframe(data)
weather_data = clean_dataframe(weather_data)
weather_data = prepare_model_data(weather_data)

In [72]:
today = dt.datetime.now().strftime("%Y-%m-%d")
file_name = f"weather_data_{today}.csv"
weather_data.to_csv(file_name, index=False)

In [73]:
# Create a Boto3 S3 client
s3_client = boto3.client('s3')


# Destination S3 bucket and object key
destination_bucket = "sagemaker-studio-550199948827-61jwze5y2ut"
destination_object_key = f"metoffice_data/{file_name}"

# Upload the file to S3
#s3_client.upload_file(file_name, destination_bucket, destination_object_key)


In [74]:
s3_file_path = f"s3://{destination_bucket}/{destination_object_key}"

In [75]:
with Run(experiment_name=experiment_name, run_name=run_name, sagemaker_session=Session()) as run:
    run.log_parameters(
        {"num_train_samples": len(weather_data), "num_test_samples": 0})
    

In [76]:

class Weather(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(Weather, self).__init__()

        # Shared hidden layers
        self.shared_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # Output layers for each output
        self.temparature_layer = nn.Linear(hidden_dim, output_dim)
        self.visibility_layer = nn.Linear(hidden_dim, output_dim)
        self.wind_direction_layer = nn.Linear(hidden_dim, output_dim)
        self.wind_speed_layer = nn.Linear(hidden_dim, output_dim)
        self.weather_type_layer = nn.Linear(hidden_dim, output_dim)
        self.pressure_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, input):

        # Pass through shared hidden layer
        shared_output = self.shared_layer(input)

        # Separate output predictions
        temperature = self.temparature_layer(shared_output)
        visibility = self.visibility_layer(shared_output)
        wind_direction = self.wind_direction_layer(shared_output)
        wind_speed = self.wind_speed_layer(shared_output)
        weather_type = self.weather_type_layer(shared_output)
        pressure = self.pressure_layer(shared_output)
        
        return wind_direction, pressure, wind_speed, temperature, visibility, weather_type

In [77]:
class WeatherData(Dataset):
    def __init__(self, data):
        self.X = torch.tensor(data[['Wind Direction(compass)', 'Wind Gust(mph)',
       'Screen Relative Humidity(%)', 'Pressure(hpa)', 'Wind Speed(mph)',
       'Temperature(C)', 'Visibility(m)', 'Weather Type', 'Pressure Tendency',
       'Dew Point(C)', 'Minutes Since 12o Clock', 'day', 'month', 'year']].values, dtype=torch.float32)
        self.y = torch.tensor(data[['Wind Direction(compass) (t+1)', 'Pressure(hpa) (t+1)',
       'Wind Speed(mph) (t+1)', 'Temperature(C) (t+1)', 'Visibility(m) (t+1)',
       'Weather Type (t+1)']].values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [78]:
batch_size = 1
num_batches = 1
num_epochs = 100

with load_run(experiment_name=experiment_name, run_name=run_name, sagemaker_session=Session()) as run:
    run.log_parameters(
        {"Batch_size": batch_size, "Epochs": num_epochs})
    

weather_data_class = WeatherData(weather_data)
weather_data_loader = DataLoader(weather_data_class, batch_size=batch_size, shuffle=True)

INFO:sagemaker.experiments.run:The run (experiment-train-run) under experiment (weatherapp-training--1692051401-5cd4) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


In [79]:
# Define the input and output dimensions
input_dim = 14
output_dim = 1
hidden_dim = 20
learning_rate = 0.001

# Create an instance of the model
model = Weather(input_dim, output_dim, hidden_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

res = str(optimizer)
lines = res.split('\n')
optim_str = re.sub(r'[^a-zA-Z]', '', lines[0])

with load_run(experiment_name=experiment_name, run_name=run_name, sagemaker_session=Session()) as run:
    run.log_parameters(
        {"Criterion": criterion, "Optimiser": optim_str, "learning_rate":learning_rate, "input_dim":input_dim, "output_dim":output_dim, "hidden_dim":hidden_dim})

INFO:sagemaker.experiments.run:The run (experiment-train-run) under experiment (weatherapp-training--1692051401-5cd4) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


In [83]:

for epoch in range(num_epochs):
    for idx, (X, y) in enumerate(weather_data_loader):

        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        wind_direction, pressure, wind_speed, temperature, visibility, weather_type = model(X)

        # Compute loss
        loss_wind_direction = criterion(wind_direction[0][0], y[0,0])
        loss_pressure = criterion(pressure[0][0], y[0,1])
        loss_wind_speed = criterion(wind_speed[0][0], y[0,2])
        loss_temperature = criterion(temperature[0][0], y[0,3])
        loss_visibility = criterion(visibility[0][0], y[0,4])
        loss_weather_type = criterion(weather_type[0][0], y[0,5])
        #total loss
        loss = loss_wind_direction + loss_pressure + loss_wind_speed + loss_temperature + loss_visibility + loss_weather_type

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Print progress
        if idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{idx+1}/{num_batches}], Loss: {loss.item():.4f}")
            with load_run(experiment_name=experiment_name, run_name=run_name, sagemaker_session=Session()) as run:
                run.log_metric(name="Train loss", value=f"{loss.item():.4f}", step=epoch+1)
                


Epoch [1/100], Batch [1/1], Loss: 277398272.0000


INFO:sagemaker.experiments.run:The run (experiment-train-run) under experiment (weatherapp-training--1692051401-5cd4) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Epoch [2/100], Batch [1/1], Loss: 214625104.0000


INFO:sagemaker.experiments.run:The run (experiment-train-run) under experiment (weatherapp-training--1692051401-5cd4) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Epoch [3/100], Batch [1/1], Loss: 718285120.0000


...

INFO:sagemaker.experiments.run:The run (experiment-train-run) under experiment (weatherapp-training--1692051401-5cd4) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Epoch [99/100], Batch [1/1], Loss: 2016452.2500


INFO:sagemaker.experiments.run:The run (experiment-train-run) under experiment (weatherapp-training--1692051401-5cd4) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Epoch [100/100], Batch [1/1], Loss: 41561696.0000


INFO:sagemaker.experiments.run:The run (experiment-train-run) under experiment (weatherapp-training--1692051401-5cd4) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.
